In [1]:
from CANCER import *


In [16]:
import torch_geometric.utils as TGUtils
import torch_geometric.transforms as TGTransforms
import networkx as nx
from matplotlib import pyplot as plt

class DataShower:
    def __init__(self):
        None
    @classmethod
    def show_dataset(self, dataset):
        print(f'Dataset: {dataset}')
        print('-------------------')
        print(f'Number of graphs: {len(dataset)}')
        print(f'Number of nodes: {dataset[0].x.shape[0]}')
        print(f'Number of features: {dataset.num_features}')
        print(f'Numer of edge attr: {dataset.edge_attr}')
        print(f'Number of classes: { dataset.num_classes}')
    
    @classmethod
    def show_y_number(self, dataset):
        y_dict = dict()
        for y in dataset.y:
            if y.item() in y_dict.keys():
                y_dict[y.item()] += 1
            else:
                y_dict[y.item()] = 1
        li = sorted(y_dict.items(), key=lambda x: x[1], reverse=True)
        print(li)
    
    @classmethod
    def path_similarity(self, dataset1, dataset2):
        # path1 = [(edge[0], edge[1]) for edge in dataset1.edge_index]
        # path2 = [(edge[0], edge[1]) for edge in dataset2.edge_index]
        for start in dataset1.edge_index[0]:
            print(start)
        
        for edge in dataset2.edge_index:
            print(edge)

        # print(path1)
        # print(path2)
    
    @classmethod
    def network_node(self, G):
        zero_degree_nodes = [node for node in G.nodes() if G.degree(node) == 0]
        zero_indegree_nodes = [node for node in G.nodes() if G.in_degree(node) == 0]
        zero_outdegree_nodes = [node for node in G.nodes() if G.out_degree(node) == 0]
        path_start_nodes = [node for node in G.nodes() if G.in_degree(node) == 0 and G.degree[node] != 0]
        path_end_nodes = [node for node in G.nodes() if G.out_degree(node) == 0 and G.degree[node] != 0]
        
        print('0-Degree  nodes#: ', len(zero_degree_nodes))
        print("0-Indegree node#:", len(zero_indegree_nodes))
        print("0-Outdegre node#:", len(zero_outdegree_nodes))
        print(f"path 시작 노드: {len(path_start_nodes)}개 {path_start_nodes}")
        print(f"path 종결 노드: {len(path_end_nodes)}개 {path_end_nodes}")
    
    @classmethod
    def network_show(self, G, mapping):
        nx.relabel_nodes(G, mapping = mapping, copy=False)
        d = dict(G.degree)
        center = [key for key, degree in enumerate(d.values()) if degree != 0]
        bound = [key for key, degree in enumerate(d.values()) if degree == 0]
        pos = nx.shell_layout(G, [center, bound], scale=10)
        label_pos = {mapping[key] : value for key, value in pos.items()}
        # print(label_pos)
        nx.draw_networkx(
            G, pos=label_pos, 
            node_color='r', 
            node_size=[v * 50 for v in d.values()],
            edge_color='b',width=0.5,
            font_size=5
        )
    @classmethod
    def show_reduced_network(self, G, mapping):
        nx.relabel_nodes(G, mapping = mapping, copy=False)
        d = dict(G.degree)
        pos = nx.circular_layout(G)
        nx.draw_networkx(
            G, pos=pos, 
            node_color='r', 
            node_size=[v * 50 for v in d.values()],
            edge_color='b',width=0.5,
            font_size=5
        )            

In [17]:
class DatasetTransformer:
    @classmethod
    def to_graph(self, dataset):
        selected = dataset[0]
        return TGUtils.to_networkx(selected, to_undirected=False)
    
    @classmethod
    def to_linked_graph(self, dataset):
        G = DatasetTransformer.to_graph(dataset)
        G.remove_nodes_from(list(nx.isolates(G)))
        return G
    
    @classmethod
    def get_largest_and_drop(self, dataset, mapping):
        import  queue
        import  math
        G = DatasetTransformer.to_graph(dataset)

        roots = [root for root in G.nodes() if G.in_degree(root) == 0]
        
        queue = queue.Queue()
        connected_nodes = [[] for _ in range(len(roots))]
        for idx, root in enumerate(roots):
            visited = dict()
            
            queue.put(root)
            connected_nodes[idx].append(root)
            visited[root] = True
            while not queue.empty():
                cur = queue.get()
                for node in G.neighbors(cur):
                    if node not in visited:
                        queue.put(node)
                        visited[node] = True
                        connected_nodes[idx].append(node)
        
        important = [connection for connection in connected_nodes if len(connection) > 1]
        row_len = 3
        col_len = math.ceil(len(important) / row_len)
        fig, axs = plt.subplots(row_len, col_len, figsize=(8 * col_len, 6 * row_len))
        
        for idx, nodes in enumerate(important):
            subgraph = G.subgraph(nodes)
            subgraph = subgraph.to_directed()
            nx.relabel_nodes(subgraph, mapping = mapping, copy=False)
            pos = nx.planar_layout(subgraph)
        
            nx.draw_networkx(
                subgraph, pos=pos, 
                # node_color='r', 
                node_size=300,
                arrowsize=10,
                # edge_color='b',width=0.5,
                font_size=15,
                ax=axs[idx % row_len, idx // row_len]
            )
        plt.tight_layout()
        plt.show()
        # nx.draw(subgraph, with_labels=True)
        # print(subgraph)


In [21]:

BreastCancer = MyDataSet(NLabel.BRAC_TCGA.filename, [ELabel.BREAST_CANCER], drop_y_threshold= 10)
Giloma = MyDataSet(NLabel.LGG_TCGA.filename, [ELabel.GILOMA], drop_y_threshold= 1)

BreastCancer_dataset = BreastCancer.make_dataset()
Giloma_dataset = Giloma.make_dataset()

DataShower.show_dataset(BreastCancer_dataset)
DataShower.show_dataset(Giloma_dataset)
# DataShower.path_similarity(BreastCancer_dataset, Giloma_dataset)


yset  5
yset  6
Dataset: tensor([[    53,     53,     40,  ..., 189329, 189314, 189281],
        [    70,     60,     75,  ..., 189229, 189329, 189299]])
-------------------
Number of graphs: 2


AttributeError: 'Tensor' object has no attribute 'x'